# Exemple d'utilisation du pricer

In [1]:
import concurrent.futures
import time
import pandas as pd
import datetime as dt
import sys
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from Classes_Both.module_enums import TypeBarriere, DirectionBarriere, ConventionBaseCalendaire, MethodeCalcul, RegType
from Classes_Both.module_marche import DonneeMarche
from Classes_Both.module_option import Option
from Classes_TrinomialTree.module_barriere import Barriere
from Classes_TrinomialTree.module_arbre_noeud import Arbre
from Classes_Both.module_pricing_analysis import StrikeComparison, VolComparison, RateComparison
from Classes_Both.module_black_scholes import BlackAndScholes
from Classes_TrinomialTree.module_grecques_empiriques import GrecquesEmpiriques

from Classes_MonteCarlo_LSM.module_brownian import Brownian
from Classes_MonteCarlo_LSM.module_LSM import LSM_method
from Classes_MonteCarlo_LSM.module_graph import LSMGraph

from Classes_Both.derivatives import OptionDerivatives, OptionDerivativesParameters


## Input et Création des objets

In [2]:
date_pricing = dt.date(2024, 1, 13)
spot = 100
volatite = 0.2
risk_free_rate = 0.02
dividende_ex_date = dt.date.today()
dividende_montant = 0
maturite = dt.date(2024, 10, 23)
strike = 101
option_exercice = 'Européenne' 
option_type = "Call"
convention_base_calendaire = 365
parametre_alpha = 3
pruning = True
epsilon_arbre = 1e-15
nb_pas = 100
nb_chemin = 100000
seed_choice = 42
antithetic_choice = False
poly_degree = 2
regress_method = "Polynomial"
barriere = Barriere(niveau_barriere=0, type_barriere=None, direction_barriere=None)
donnee_marche = DonneeMarche(date_pricing, spot, volatite, risk_free_rate, risk_free_rate, dividende_ex_date, dividende_montant)
option = Option(maturite, strike, barriere=barriere, 
                americaine=False if option_exercice == 'Européenne' else True, call=True if option_type == "Call" else False,
                date_pricing=date_pricing)

pricer_arbre = Arbre(nb_pas=nb_pas, donnee_marche=donnee_marche, option=option, convention_base_calendaire=convention_base_calendaire, parametre_alpha=parametre_alpha, pruning=pruning, epsilon=epsilon_arbre)
pricer_arbre.pricer_arbre()

brownian = Brownian(time_to_maturity=(maturite-date_pricing).days / convention_base_calendaire, nb_step=nb_pas, nb_trajectoire=nb_chemin, seed=seed_choice)
pricer_LSM = LSM_method(option)

bns = BlackAndScholes(modele=pricer_arbre)


### Prix de l'arbre

In [3]:
pricer_arbre.prix_option

7.290791114447453

### Prix de black scholes

In [4]:
bns.bs_pricer()

7.292591439771627

### Prix de LSM

In [5]:
price, std_error, intevalles = pricer_LSM.LSM(brownian, donnee_marche, 
                                            method= 'vector',
                                            antithetic = antithetic_choice,
                                            poly_degree=poly_degree, 
                                            model_type=regress_method)
price, std_error, intevalles

(7.3010821280856515,
 0.03672867462542061,
 (7.2276247788348105, 7.3745394773364925))

### Graphique

In [6]:
graph = LSMGraph(option=option, market=donnee_marche)


In [7]:
# graph.afficher_mouvements_browniens(brownian,nb_trajectoires=nb_chemin)
# trajectoire = pricer_LSM.Price(market=donnee_marche, brownian=brownian)
# graph_spot = graph.afficher_trajectoires_prix(trajectoire,brownian,nb_trajectoires=nb_chemin)
# graph_spot.show()

In [8]:
graph_vs = graph.comparer_methodes()
graph_vs.show()

In [7]:
graph_seed = graph.comparer_seeds()
graph_seed.show()
            

In [10]:
graph_nb_pas = graph.comparer_steps()
graph_nb_pas.show()
    

In [11]:
graph_path = graph.comparer_convergence_paths()
graph_path.show()

Traitement de la méthode: scalar
Traitement de la méthode: vector


In [12]:
graph_poly, df_results = graph.comparer_polynomes()
graph_poly.show()


In [13]:
graph_degree, df_results = graph.comparer_degres_par_type()
graph_degree.show()

In [31]:
# Définir les plages de valeurs à tester
max_cpu = 4  # Nombre de cœurs à utiliser
step_list = [20]  # Liste de pas à tester
strike_list = np.arange(spot-10, spot+10, 1.0)  # Variation du strike

# Créer l'objet de comparaison
strike_comparison = StrikeComparison(max_cpu, step_list, strike_list, donnee_marche, brownian, option)

# Afficher le graphique
fig_strike = strike_comparison.graph_strike_comparison()
fig_strike.show()

# Afficher le tableau de résultats si nécessaire
strike_comparison.results_df.sort_values(by='Strike', ascending=True)

,Nombre de pas,Prix arbre trinomial,Temps arbre,Diff arbre - B&S,Prix LSM,Temps LSM,Diff LSM - B&S,Diff arbre * nb pas,Diff LSM * nb pas,Strike
1,20,13.806520,0.336195,-0.009743,13.864497,1.133416,0.048233,-0.974291,0.964668,90.0
0,20,13.119296,0.340238,0.006224,13.113677,1.133472,0.000605,0.622363,0.012097,91.0
3,20,12.432071,0.330755,0.000988,12.379110,1.159332,-0.051973,0.098805,-1.039464,92.0
2,20,11.765202,0.336195,-0.005529,11.795976,1.127701,0.025245,-0.552908,0.504905,93.0
4,20,11.139964,0.277495,0.007595,11.124970,1.005795,-0.007400,0.759486,-0.147990,94.0
6,20,10.514726,0.277495,-0.001547,10.548703,1.015451,0.032429,-0.154696,0.648587,95.0
5,20,9.919294,0.257145,-0.003344,9.947962,1.017052,0.025325,-0.334364,0.506490,96.0
7,20,9.359858,0.299682,0.008284,9.388172,1.000474,0.036598,0.828437,0.731967,97.0
8,20,8.800423,0.302024,-0.002694,8.807591,0.954034,0.004474,-0.269450,0.089483,98.0
9,20,8.274087,0.300015,-0.003137,8.299051,0.962734,0.021827,-0.313696,0.436532,99.0


In [32]:
# Définir les plages de valeurs à tester pour la volatilité
vol_list = np.arange(0.1, 0.5, 0.02)  # Variation de volatilité entre 10% et 50%

# Créer l'objet de comparaison
vol_comparison = VolComparison(max_cpu, step_list, vol_list, donnee_marche, brownian, option)

# Afficher le graphique
fig_vol = vol_comparison.graph_vol()
fig_vol.show()

# Afficher le tableau de résultats
vol_comparison.results_df.sort_values(by='Volatilité', ascending=True)

,Nombre de pas,Prix arbre trinomial,Temps arbre,Diff arbre - B&S,Prix LSM,Temps LSM,Diff LSM - B&S,Diff arbre * nb pas,Diff LSM * nb pas,Volatilité
2,20,3.798353,0.315959,0.003436,3.806685,0.990173,0.011768,0.343628,0.235369,0.10
1,20,4.497601,0.324168,0.002859,4.449729,0.974987,-0.045013,0.285898,-0.900253,0.12
3,20,5.196522,0.324168,0.001945,5.243465,0.987823,0.048888,0.194502,0.977750,0.14
0,20,5.895064,0.315959,0.000821,5.896324,0.969362,0.002082,0.082134,0.041632,0.16
4,20,6.593171,0.264933,-0.000441,6.618975,0.949274,0.025362,-0.044131,0.507249,0.18
5,20,7.290791,0.268876,-0.001800,7.278287,0.956533,-0.014304,-0.180033,-0.286086,0.20
6,20,7.987870,0.263497,-0.003228,8.053403,0.962680,0.062305,-0.322842,1.246094,0.22
7,20,8.684353,0.270074,-0.004707,8.697262,0.957807,0.008201,-0.470727,0.164028,0.24
8,20,9.380188,0.274497,-0.006224,9.455006,0.938720,0.068594,-0.622406,1.371881,0.26
9,20,10.075322,0.286688,-0.007770,10.170851,0.942380,0.087759,-0.776953,1.755186,0.28


In [34]:
# Définir les plages de valeurs à tester pour le taux
rate_list = np.arange(-0.5,0.5, 0.01)  # Variation du taux entre -2% et 10%

# Créer l'objet de comparaison
rate_comparison = RateComparison(max_cpu, step_list, rate_list, donnee_marche, brownian, option)

# Afficher le graphique
fig_rate = rate_comparison.graph_rate()
fig_rate.show()

# Afficher le tableau de résultats
rate_comparison.results_df.sort_values(by='Taux d\'intérêt', ascending=True)

,Nombre de pas,Prix arbre trinomial,Temps arbre,Diff arbre - B&S,Diff arbre * nb pas,Prix LSM,Temps LSM,Diff LSM - B&S,Diff LSM * nb pas,Taux d'intérêt
3,20,0.087754,0.024882,-0.000153,-0.003058,0.084430,1.081014,-0.003476,-0.069522,-0.50
0,20,0.102553,0.021368,0.003135,0.062703,0.094582,1.084528,-0.004836,-0.096725,-0.49
1,20,0.117239,0.022803,0.004984,0.099674,0.121769,1.077692,0.009514,0.190284,-0.48
2,20,0.131810,0.019914,0.005266,0.105324,0.129275,1.083210,0.002731,0.054625,-0.47
6,20,0.146268,0.019127,0.003847,0.076940,0.140507,0.960577,-0.001914,-0.038285,-0.46
...,...,...,...,...,...,...,...,...,...,...
95,20,28.979183,0.017968,-0.009412,-0.188236,29.045727,1.040632,0.057132,1.142639,0.45
96,20,29.516853,0.016113,-0.006040,-0.120795,29.577289,1.058088,0.054397,1.087942,0.46
97,20,30.053211,0.017968,-0.001471,-0.029415,29.990067,1.040632,-0.064615,-1.292292,0.47
98,20,30.585412,0.020214,0.001573,0.031468,30.550162,0.947877,-0.033677,-0.673532,0.48
